In [1]:
# import os
# from io import BytesIO
# import tarfile
# import tempfile
# from six.moves import urllib

# from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
# from PIL import Image

import tensorflow as tf
print(tf.__version__)

from deeplab.core import resnet_v1_beta
from deeplab.core import feature_extractor
from deeplab import common
from deeplab import model

from deeplab.core import utils
from deeplab.core import conv2d_ws
# from tensorflow.contrib.slim.nets import resnet_utils
import functools
from tensorflow.contrib import slim as contrib_slim  ## this is the tf-slim (contrib.slim)
slim = contrib_slim
from slim.nets import resnet_utils  ## the is the slim image classification library from model/research/slim

from lib import dataloader

%load_ext autoreload
%autoreload 2

1.12.3


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')

## set parameters

In [3]:
crop_size = [256, 256]
outputs_to_num_classes = {'semantic': 3}

model_options = common.ModelOptions(
    outputs_to_num_classes,
    crop_size,
    output_stride=8
)._replace(
    add_image_level_feature=True,
    aspp_with_batch_norm=True,
    aspp_with_separable_conv=False,
    decoder_use_separable_conv=False,
    logits_kernel_size=1,
    decoder_output_stride=[4,2],
    multi_grid=[1,2],
    atrous_rates=[2,4,6],
    model_variant='resnet_mod') 

for k, v in model_options._asdict().items():
    print(k, v)

outputs_to_num_classes {'semantic': 3}
crop_size [256, 256]
atrous_rates [2, 4, 6]
output_stride 8
preprocessed_images_dtype <dtype: 'float32'>
merge_method max
add_image_level_feature True
image_pooling_crop_size None
image_pooling_stride [1, 1]
aspp_with_batch_norm True
aspp_with_separable_conv False
multi_grid [1, 2]
decoder_output_stride [4, 2]
decoder_use_separable_conv False
logits_kernel_size 1
model_variant resnet_mod
depth_multiplier 1.0
divisible_by None
prediction_with_upsampled_logits True
dense_prediction_cell_config None
nas_architecture_options {'nas_stem_output_num_conv_filters': 20, 'nas_use_classification_head': False, 'nas_remove_os32_stride': False}
use_bounded_activation False
aspp_with_concat_projection True
aspp_with_squeeze_and_excitation False
aspp_convs_filters 256
decoder_use_sum_merge False
decoder_filters 256
decoder_output_is_logits False
image_se_uses_qsigmoid False
label_weights 1.0
sync_batch_norm_method None
batch_norm_decay 0.9997


In [4]:
weight_decay=0.0001
is_training=True
fine_tune_batch_norm = True
nas_training_hyper_parameters = None

output_stride=model_options.output_stride,
multi_grid=model_options.multi_grid,
depth_multiplier=model_options.depth_multiplier,
divisible_by=model_options.divisible_by,
weight_decay=weight_decay,
is_training=is_training,
preprocessed_images_dtype=model_options.preprocessed_images_dtype,
fine_tune_batch_norm=fine_tune_batch_norm,
nas_architecture_options=model_options.nas_architecture_options,
nas_training_hyper_parameters=nas_training_hyper_parameters,
use_bounded_activation=model_options.use_bounded_activation

In [5]:
preprocess_images=True,
preprocessed_images_dtype=tf.float32
global_pool=False
num_classes = None
name_scope = {
    'mobilenet_v2': 'MobilenetV2',
    'mobilenet_edgetpu': 'MobilenetEdgeTPU',
    'mobilenet_v3_large_seg': 'MobilenetV3',
    'mobilenet_v3_small_seg': 'MobilenetV3',
    'resnet_v1_18': 'resnet_v1_18',
    'resnet_v1_18_beta': 'resnet_v1_18',
    'resnet_v1_50': 'resnet_v1_50',
    'resnet_v1_50_beta': 'resnet_v1_50',
    'resnet_v1_101': 'resnet_v1_101',
    'resnet_v1_101_beta': 'resnet_v1_101',
    'xception_41': 'xception_41',
    'xception_65': 'xception_65',
    'xception_71': 'xception_71',
    'nas_pnasnet': 'pnasnet',
    'nas_hnasnet': 'hnasnet',
    'resnet_mod' : 'resnet_mod',
    'resnet_v2_50' : 'resnet_v2_50',
}
model_variant = model_options.model_variant

## define custom resnet feature extractor

In [6]:
# from deeplab.core import resnet_v1_beta
# from deeplab.core import utils
# from deeplab.core import conv2d_ws
# from tensorflow.contrib.slim.nets import resnet_utils
# import functools

# def resnet_mod_a(inputs,
#                num_classes=None,
#                is_training=None,
#                global_pool=False,
#                output_stride=None,
#                multi_grid=None,
#                root_depth_multiplier=0.25,
#                reuse=None,
#                scope='resnet_123'):
#     """
#     A custom Resnet v1 18 beta variant.
#     """
    
#     ## define the multi_grid/atrous blocks
#     if multi_grid is None:
#         multi_grid = [1,1]
#     else:
#         if len(multi_grid) != 2:
#           raise ValueError('Expect multi_grid to have length 2.')

#     block4_args = []
#     for rate in multi_grid:
#         block4_args.append({'depth': 512, 'stride': 1, 'unit_rate': rate})

#     blocks = [
#       resnet_v1_beta.resnet_v2_small_beta_block(
#           'block1', base_depth=64, num_units=1, stride=2),
#       resnet_v1_beta.resnet_v2_small_beta_block(
#           'block2', base_depth=128, num_units=1, stride=2),
#       resnet_v1_beta.resnet_v2_small_beta_block(
#           'block3', base_depth=256, num_units=1, stride=2),
#       resnet_utils.Block('block4', resnet_v1_beta.lite_bottleneck_v2, block4_args),
#     ]
    
#     root_block_fn = functools.partial(conv2d_ws.conv2d_same,
#                                       num_outputs=32,
#                                       kernel_size=3,
#                                       stride=1,
#                                       scope='root_conv1')

#     batch_norm = utils.get_batch_norm_fn(sync_batch_norm_method)
#     with tf.variable_scope(scope, 'bbb', [inputs], reuse=reuse) as sc:
#         end_points_collection = sc.original_name_scope + '_end_points'
#         with slim.arg_scope([
#             conv2d_ws.conv2d, resnet_utils.stack_blocks_dense, resnet_v1_beta.lite_bottleneck_v2 
#             ],outputs_collections=end_points_collection):
#             if is_training is not None:
#                 arg_scope = slim.arg_scope([batch_norm], is_training=is_training)
#             else:
#                 arg_scope = slim.arg_scope([])
#             with arg_scope:
#                 print(arg_scope)
#                 net = inputs
#                 if output_stride is not None:
#                   if output_stride % 4 != 0:
#                     raise ValueError('The output_stride needs to be a multiple of 4.')
#     #               output_stride //= 4
#                 net = root_block_fn(net)
#     #             net = slim.max_pool2d(net, 3, stride=2, padding='SAME', scope='pool1')
#                 net = resnet_utils.stack_blocks_dense(net, blocks, output_stride)
#                 ## add a batchnorm and relu layer since the last conv output don't have them in v2
#                 net = slim.batch_norm(net, activation_fn=tf.nn.relu, scope='postnorm')
#                 # Convert end_points_collection into a dictionary of end_points.
#                 end_points = slim.utils.convert_collection_to_dict(
#                     end_points_collection)

#                 return net, end_points

In [7]:
arg_scope = resnet_utils.resnet_arg_scope(
        weight_decay=weight_decay,
        batch_norm_decay=0.95,
        batch_norm_epsilon=1e-5,
        batch_norm_scale=True)

def _preprocess_zero_mean_unit_range(inputs, dtype=tf.float32):
  """Map image values from [0, 255] to [-1, 1]."""
  preprocessed_inputs = (2.0 / 255.0) * tf.to_float(inputs) - 1.0
  return tf.cast(preprocessed_inputs, dtype=dtype)


func = resnet_v1_beta.resnet_mod
# func = resnet_mod_a
preprocess_function = _preprocess_zero_mean_unit_range

@functools.wraps(func)
def network_fn(inputs, *args, **kwargs):
    with slim.arg_scope(arg_scope):
        return func(preprocess_function(inputs, preprocessed_images_dtype),
              *args, **kwargs)

## create input and extract features

In [8]:
inputs = tf.keras.Input(shape=(crop_size[0], crop_size[1], 1), name='input_layer')
with tf.variable_scope('resnet_mod'):
    features, end_points = network_fn(inputs=inputs,
                                  num_classes=num_classes,
                                  is_training=True,
                                  global_pool=global_pool,
                                  output_stride=model_options.output_stride,
                                  multi_grid=model_options.multi_grid,
                                  reuse=tf.AUTO_REUSE,
                                  scope=name_scope[model_variant])

## setup ASPP block

In [9]:
reuse = tf.AUTO_REUSE
slim = contrib_slim

LOGITS_SCOPE_NAME = 'logits'
MERGED_LOGITS_SCOPE = 'merged_logits'
IMAGE_POOLING_SCOPE = 'image_pooling'
ASPP_SCOPE = 'aspp'
CONCAT_PROJECTION_SCOPE = 'concat_projection'
DECODER_SCOPE = 'decoder'
META_ARCHITECTURE_SCOPE = 'meta_architecture'

PROB_SUFFIX = '_prob'

_resize_bilinear = utils.resize_bilinear
scale_dimension = utils.scale_dimension
split_separable_conv2d = utils.split_separable_conv2d


batch_norm_params = utils.get_batch_norm_params(
                      decay=0.9997,
                      epsilon=1e-5,
                      scale=True,
                      is_training=(is_training and fine_tune_batch_norm),
#                       is_training=None,
                      sync_batch_norm_method=model_options.sync_batch_norm_method)
batch_norm = utils.get_batch_norm_fn(
                      model_options.sync_batch_norm_method)
activation_fn = tf.nn.relu

batch_norm_params['is_training'] = tf.placeholder(tf.bool, name='training')

In [10]:
with slim.arg_scope(
          [slim.conv2d, slim.separable_conv2d],
          weights_regularizer=slim.l2_regularizer(weight_decay),
          activation_fn=activation_fn,
          normalizer_fn=batch_norm,
          padding='SAME',
          stride=1,
          reuse=reuse):
    with slim.arg_scope([batch_norm], **batch_norm_params):
        depth = model_options.aspp_convs_filters
        branch_logits = []

        if model_options.add_image_level_feature:
            if model_options.crop_size is not None:
                image_pooling_crop_size = model_options.image_pooling_crop_size
              # If image_pooling_crop_size is not specified, use crop_size.
                if image_pooling_crop_size is None:
                    image_pooling_crop_size = model_options.crop_size
                pool_height = scale_dimension(
                                image_pooling_crop_size[0],
                                1. / model_options.output_stride)
                pool_width = scale_dimension(
                                image_pooling_crop_size[1],
                                1. / model_options.output_stride)
                image_feature = slim.avg_pool2d(
                                    features, [pool_height, pool_width],
                                    model_options.image_pooling_stride, padding='VALID')
                resize_height = scale_dimension(
                                  model_options.crop_size[0],
                                  1. / model_options.output_stride)
                resize_width = scale_dimension(
                                  model_options.crop_size[1],
                                  1. / model_options.output_stride)
            else:
              # If crop_size is None, we simply do global pooling.
                pool_height = tf.shape(features)[1]
                pool_width = tf.shape(features)[2]
                image_feature = tf.reduce_mean(
                                    features, axis=[1, 2], keepdims=True)
                resize_height = pool_height
                resize_width = pool_width
            image_feature_activation_fn = tf.nn.relu
            image_feature_normalizer_fn = batch_norm
            
            
            image_feature = slim.conv2d(
                                image_feature, depth, 1,
                                activation_fn=image_feature_activation_fn,
                                normalizer_fn=image_feature_normalizer_fn,
                                scope=IMAGE_POOLING_SCOPE)
            image_feature = _resize_bilinear(
                                image_feature,
                                [resize_height, resize_width],
                                image_feature.dtype)
            # Set shape for resize_height/resize_width if they are not Tensor.
            if isinstance(resize_height, tf.Tensor):
              resize_height = None
            if isinstance(resize_width, tf.Tensor):
              resize_width = None
            image_feature.set_shape([None, resize_height, resize_width, depth])
            if not model_options.aspp_with_squeeze_and_excitation:
                branch_logits.append(image_feature)

        # Employ a 1x1 convolution.
        branch_logits.append(slim.conv2d(features, depth, 1,
                                       scope=ASPP_SCOPE + str(0)))
        if model_options.atrous_rates:
            # Employ 3x3 convolutions with different atrous rates.
            for i, rate in enumerate(model_options.atrous_rates, 1):
                scope = ASPP_SCOPE + str(i)
                if model_options.aspp_with_separable_conv:
                    aspp_features = split_separable_conv2d(
                                        features,
                                        filters=depth,
                                        rate=rate,
                                        weight_decay=weight_decay,
                                        scope=scope)
                else:
                    aspp_features = slim.conv2d(
                        features, depth, 3, rate=rate, scope=scope)
                branch_logits.append(aspp_features)
        
        # Merge branch logits.
        concat_logits = tf.concat(branch_logits, 3)
        if model_options.aspp_with_concat_projection:
            concat_logits = slim.conv2d(
                            concat_logits, depth, 1, scope=CONCAT_PROJECTION_SCOPE)
            concat_logits = slim.dropout(
                            concat_logits,
                            keep_prob=0.9,
#                             is_training=is_training,
                            scope=CONCAT_PROJECTION_SCOPE + '_dropout')

In [11]:
pool_width, features.shape, model_options.output_stride

(32,
 TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(512)]),
 8)

## build decoder part

In [12]:
# model_options = model_options._replace(
# decoder_output_stride = [4])

In [13]:
model_options.decoder_output_stride, model_options.crop_size, model_options.decoder_use_separable_conv, model_options.decoder_use_sum_merge

([4, 2], [256, 256], False, False)

In [14]:
model_options.decoder_filters, model_options.decoder_output_is_logits, model_options.model_variant, model_options.outputs_to_num_classes

(256, False, 'resnet_mod', {'semantic': 3})

In [15]:
model_options.atrous_rates, model_options.logits_kernel_size, model_options.prediction_with_upsampled_logits

([2, 4, 6], 1, True)

In [16]:
is_training=True
training = tf.placeholder(tf.bool, name='training')

In [17]:
crop_size = model_options.crop_size
    
decoded = model.refine_by_decoder(
            concat_logits,
            end_points,
            crop_size=crop_size,
            decoder_output_stride=model_options.decoder_output_stride,
            decoder_use_separable_conv=model_options.decoder_use_separable_conv,
            decoder_use_sum_merge=model_options.decoder_use_sum_merge,
            decoder_filters=model_options.decoder_filters,
            decoder_output_is_logits=model_options.decoder_output_is_logits,
            model_variant=model_options.model_variant,
            weight_decay=weight_decay,
            reuse=reuse,
            is_training=is_training,
            fine_tune_batch_norm=False,
            use_bounded_activation=model_options.use_bounded_activation)

length feature list:1
length decoder feature list:1
length decoder feature list:2
(?, 64, 64, 256)
length feature list:1
length decoder feature list:1
length decoder feature list:2
(?, 128, 128, 256)


In [18]:
decoded.shape

TensorShape([Dimension(None), Dimension(128), Dimension(128), Dimension(256)])

In [19]:
model_options.outputs_to_num_classes

{'semantic': 3}

In [21]:
outputs = model.get_branch_logits(
                              decoded,
                              model_options.outputs_to_num_classes['semantic'],
                              model_options.atrous_rates,
                              aspp_with_batch_norm=model_options.aspp_with_batch_norm,
                              kernel_size=model_options.logits_kernel_size,
                              weight_decay=weight_decay,
                              reuse=reuse,
                              scope_suffix='')
outputs.shape

TensorShape([Dimension(None), Dimension(128), Dimension(128), Dimension(3)])

In [22]:
from deeplab.core.utils import resize_bilinear

outputs_scaled = resize_bilinear(outputs,
                        crop_size,
                        tf.float32)
# MERGED_LOGITS_SCOPE = 'merged_logits'
# outputs_to_scales_to_logits = {
#       k: {}
#       for k in model_options.outputs_to_num_classes
# }
# outputs_to_scales_to_logits[output][MERGED_LOGITS_SCOPE] = outputs_to_logits[output]

# scales_to_logits = outputs_to_scales_to_logits[output]
# logits = scales_to_logits[MERGED_LOGITS_SCOPE]
# logits = resize_bilinear(outputs_to_logits[output],
#                         tf.shape(inputs)[1:3],
#                         scales_to_logits[MERGED_LOGITS_SCOPE].dtype)
# predictions = tf.argmax(logits, 3)
# predictions = tf.nn.softmax(logits)

## run session graph, write summary for tensorboard

In [23]:
g = tf.get_default_graph()
with g.as_default():
    with tf.Session(graph=g) as sess:
        img = tf.random_uniform(
                (4, crop_size[0], crop_size[1], 1))
        print(img.shape)
        print(img.dtype)
#         outputs_to_scales_to_logits = model.multi_scale_logits(
#                                         inputs,
#                                         model_options,
#                                         image_pyramid=[1.0])

        sess.run(tf.global_variables_initializer())
#         outputs_to_scales_to_logits = sess.run(outputs_to_logits, feed_dict={'input_layer:0': img.eval(), 'training:0':True})
        outputs = sess.run(outputs_scaled, feed_dict={'input_layer:0': img.eval(), 'training:0':True})

        writer = tf.summary.FileWriter("output2/resnet_mod_v2_decode4-2", sess.graph)
        writer.close()

(4, 256, 256, 1)
<dtype: 'float32'>


In [ ]:
outputs_to_scales_to_logits['semantic'].shape